# Cleaning up google forms data

**Today we will be cleanning up some exported forms data** <br>
**We will be using the `pandas` library to do this** <br>
Pandas is a Python library used for working with data sets. It has functions for analyzing, cleaning, exploring, and manipulating data.


In [1]:
# first we import the necessary libraries
import pandas as pd
import numpy as np


# and then we load in the data(student data as std)
std = pd.read_csv('../data/raw/forms_responses_12955.csv')

lets get an overview of the data

In [3]:
std.sample(10)

,Timestamp,Student ID,Age,Gender,Department,GPA,Satisfaction (1-5),Comments
3989,09/30/2023 20:55:00,STUD8886,24,Female,Physics,2.17,1.0,NaN
15772,09/23/2023 16:15:00,STUD3639,19,Male,Zoology,0.18,1.0,NaN
245,09/06/2023 15:17:00,STUD4169,19,Malee,Physics,3.38,1.0,Comment 245: The course was great!
182,09/03/2023 11:07:00,STUD5350,24,Male,Geophysics,1.5,3.0,Comment 182: The course was great!
1057,09/08/2023 12:38:00,STUD4776,100,Male,Zoology,0.42,1.0,NaN
12682,09/23/2023 00:48:00,STUD3071,15,Male,Computer Science,2.63,1.0,NaN
11188,09/29/2023 13:10:00,STUD3428,15,Male,Zoology,3.5,3.0,NaN
7532,09/03/2023 01:02:00,STUD7072,18,Other,Geophysics,5.0,6.0,This is spam... ignore me
4885,09/26/2023 23:27:00,STUD6728,19,Male,Cell Bio,3.78,3.0,This is spam... ignore me
9428,09/09/2023 18:26:00,STUD3962,NaN,Femal,Physics,2.91,NaN,This is spam... ignore me


In [4]:
std.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Timestamp           16000 non-null  object 
 1   Student ID          16000 non-null  object 
 2   Age                 14769 non-null  object 
 3   Gender              15016 non-null  object 
 4   Department          15215 non-null  object 
 5   GPA                 14896 non-null  object 
 6   Satisfaction (1-5)  14845 non-null  float64
 7   Comments            11174 non-null  object 
dtypes: float64(1), object(7)
memory usage: 1000.1+ KB


In [5]:
std.describe()

,Satisfaction (1-5)
count,14845.000000
mean,3.103941
std,1.660700
min,0.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,7.000000


In [6]:
std.columns

Index(['Timestamp', 'Student ID', 'Age', 'Gender', 'Department', 'GPA',
       'Satisfaction (1-5)', 'Comments'],
      dtype='object')

In [7]:
std.dtypes

Timestamp              object
Student ID             object
Age                    object
Gender                 object
Department             object
GPA                    object
Satisfaction (1-5)    float64
Comments               object
dtype: object

**We discover that almost all data entries are objects, hence there are a lot of errors.** <br>
lets fix that, starting with the timestamp column

In [8]:
std.Timestamp = pd.to_datetime(std.Timestamp, errors='coerce')

In [9]:
std.dropna(subset=['Timestamp'], inplace=True)

In [10]:
#after using dropna and pd.to_datetime we check the info again
std.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15672 entries, 0 to 15999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Timestamp           15672 non-null  datetime64[ns]
 1   Student ID          15672 non-null  object        
 2   Age                 14470 non-null  object        
 3   Gender              14706 non-null  object        
 4   Department          14895 non-null  object        
 5   GPA                 14596 non-null  object        
 6   Satisfaction (1-5)  14539 non-null  float64       
 7   Comments            10927 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 1.1+ MB


**As you can see from the latest info, we have the timestamp labelled as as a TYPE datetime64[ns]**

_lets work on the student id column next_

In [11]:
# Lets work on the student id column next by adding an underscore to the column name for easy access
std.rename(columns={'Student ID': 'Student_ID'}, inplace=True)
#lets also make the id lowercase
std.Student_ID = std.Student_ID.str.lower()
# we can also remove any leading or trailing spaces
std.Student_ID = std.Student_ID.str.strip()

_lets work on the age column next_

In [12]:
# from the info above we can see that the age column is an object type which is not correct
# we can convert it to numeric using pd.to_numeric
std.Age = pd.to_numeric(std.Age, errors='coerce')

std.Age.unique()
# we can also see there are missing data and a lot oof outliers at age 100 and 15 which is impossible for a student
# we can drop the missing data and filter out the outliers
std.dropna(subset=['Age'], inplace=True)
std = std[(std.Age >= 18) & (std.Age <= 60)]

# finally we make sure the age is an integer type as age cannot be fractional
std.Age = std.Age.astype(int)

_lets work on the gender column next_

In [13]:
# List unique values to verify
print(std['Gender'].unique())

# Correct typos and standardize values
std['Gender'] = std['Gender'].replace({
    'Femal': 'Female',
    'Malee': 'Male',
    'Othr': 'Other',
    np.nan: 'Other'
})

# Ensure no remaining NaN (redundant due to replace, but kept for clarity)
std['Gender'] = std['Gender'].fillna('Other')

# Standardize case (first letter uppercase, rest lowercase)
std['Gender'] = std['Gender'].str.title()

# Convert to pandas string dtype
std['Gender'] = std['Gender'].astype('string')

# Verify dtype and values
print(std['Gender'].dtype)  # Should be: string
print(std['Gender'].unique())


['Female' 'Male' 'Femal' 'Malee' 'Other' nan 'Othr']
string
<StringArray>
['Female', 'Male', 'Other']
Length: 3, dtype: string


_lets work on the deparment columns next_

In [14]:
# List unique values to verify replacements
print(std['Department'].unique())

# Replace typos and standardize values
std['Department'] = std['Department'].replace({
    np.nan: 'Undeclared',
    'Marine Sci': 'Marine Sciences',
    'Geo': 'Geosciences',
    'Biochem': 'Biochemistry',
    'Maths': 'Mathematics',
    'Phys': 'Physics',
    'Bio': 'Biology',
    'Cell Bio': 'Cell Biology and Genetics',
    'Chem': 'Chemistry',
    'Geophy': 'Geophysics',
    'Zoo': 'Zoology',
    'Microbio': 'Microbiology',
    'Comp Sci': 'Computer Science'
})

# Ensure no remaining NaN (already handled by replace)
std['Department'] = std['Department'].fillna('Undeclared')

# Convert to pandas string dtype (not object)
std['Department'] = std['Department'].astype('string')

# Verify dtype and values
print(std['Department'].dtype)  # Should be: string
print(std['Department'].unique())

['Marine Sci' 'Zoology' 'Mathematics' 'Chemistry' 'Geophysics'
 'Computer Science' 'Botany' 'Biochemistry' 'Cell Biology and Genetics'
 'Marine Sciences' nan 'Microbiology' 'Physics' 'Biochem' 'Maths' 'Phys'
 'Geology' 'Bio' 'Cell Bio' 'Chem' 'Geophy' 'Geo' 'Zoo' 'Microbio'
 'Comp Sci']
string
<StringArray>
[          'Marine Sciences',                   'Zoology',
               'Mathematics',                 'Chemistry',
                'Geophysics',          'Computer Science',
                    'Botany',              'Biochemistry',
 'Cell Biology and Genetics',                'Undeclared',
              'Microbiology',                   'Physics',
                   'Geology',                   'Biology',
               'Geosciences']
Length: 15, dtype: string


_lets work on the GPA columns next_

In [15]:
# from the samples i can see the GPA column has some non-numeric values like A, B, C, D and F, we can check for them
std.GPA.unique()
# then we can map them to the minimum equivalent numeric value
std.GPA = std.GPA.replace({
    'A': 4.5,
    'B': 3.5,
    'C': 3.0,
    'D': 2.5,
    'F': 1.5
})

# we can then convert the gpa column to numeric
std.GPA = pd.to_numeric(std.GPA, errors='coerce')

# secondly in order to maximize the data we can fill the missing gpa values with the interpolated values
std.GPA = std.GPA.fillna(std.GPA.interpolate(method='linear'))
std.GPA = std.GPA.round(2)

# finally we remove the outliers < 1.0 in a 5.0 scale
std = std[std.GPA >= 1.0]

_lets work on the satisfaction next by doing almost exactly the same thing with the GPA coluumn_

In [ ]:
# we fill the missing values with the linear interpolation
std['Satisfaction (1-5)'] = std['Satisfaction (1-5)'].fillna(std['Satisfaction (1-5)'].interpolate(method='linear'))
std['Satisfaction (1-5)'] = std['Satisfaction (1-5)'].round(2)

# finally we remove the outliers < 1.0 in a 5.0 scale
std = std[std['Satisfaction (1-5)'] >= 1.0]

<class 'pandas.core.frame.DataFrame'>
Index: 9908 entries, 1 to 15999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Timestamp           9908 non-null   datetime64[ns]
 1   Student_ID          9908 non-null   object        
 2   Age                 9908 non-null   int64         
 3   Gender              9908 non-null   string        
 4   Department          9908 non-null   string        
 5   GPA                 9908 non-null   float64       
 6   Satisfaction (1-5)  9908 non-null   float64       
 7   Comments            6894 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2), string(2)
memory usage: 696.7+ KB


***Finally lets work on the comments column***

In [42]:
# Firstly we check for unique comments
std.Comments.unique()

# since we have no unique comments, we align them to look like each other
std['Comments'] = std['Comments'].fillna("The course was great!")

# before identifying the comments, let's arrange the dataframe by date
std = std.sort_values('Timestamp', ascending=True).reset_index(drop=True)

def comments_standardize(id, text):
    # Handle spam comments
    if isinstance(text, str) and text.startswith('This is spam'):
        return f'Comment {id}: The course was great!'
    # Handle comments with 'Comment <number>: ...' format
    elif isinstance(text, str) and text.startswith('Comment'):
        # Extract text after 'Comment <number>: '
        parts = text.split(':', 1)
        if len(parts) > 1:
            return f'Comment {id}: {parts[1].strip()}'
        return f'Comment {id}: No comment'
    # Handle empty or other comments
    elif isinstance(text, str) and text.strip():
        return f'Comment {id}: {text.strip()}'
    else:
        return f'Comment {id}: No comment'
    

# Apply the function to standardize comments
std['Comments'] = [comments_standardize(i, txt) for i, txt in enumerate(std['Comments'])]
std.Comments = std.Comments.astype('string')



**Lets then make some finishing touches to the dataframe**

In [47]:
# Unfortunantely i can't think of any feature engineering to do on this dataset so we will just save the cleaned dataframe to a new csv file
std.to_csv('../data/cleaned/cleaned_student_data.csv', index=False)
std.to_excel('../data/cleaned/cleaned_student_data.xlsx', index=False, sheet_name='Cleaned Student Data')